# 前情
食物品种共有11类：Bread, Dairy product, Dessert, Egg, Fried food, Meat, Noodles/Pasta, Rice, Seafood, Soup, and Vegetable/Fruit.

Training set: 9866张

Validation set: 3430张

Testing set: 3347张

In [1]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms #torchvision是pytorch的一个图形库
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time

# Read image 
利用 OpenCV (cv2) 讀入照片，並将照片转换成指定的分辨率，转化成数组存放在 numpy array 中

In [2]:
def readfile(path, label):# label 是0 或 1，1代表需要回傳 y 值
    image_dir = sorted(os.listdir(path))
    # uint8是专门用于存储各种图像的，范围是从0–255
    x = np.zeros((len(image_dir), 128, 128, 3), dtype=np.uint8) #初始化，
    y = np.zeros((len(image_dir)), dtype=np.uint8) #初始化
    # 给文件夹里的图片一个编号，并将编号和图片组合成一个表
    for i, file in enumerate(image_dir):
        # cv.imread(filename[, flags]) 得到图像的通道和色彩信息，使用默认参数: 8位深度，3通道
        # 结果是一个 高度X宽度 的矩阵，里面的元素是一个3维行向量，表示RGB的值
        img = cv2.imread(os.path.join(path, file)) #将文件夹位置和图片名拼接起来，得到图片位置
        x[i, :, :] = cv2.resize(img,(128, 128)) # 图片分辨率缩放成 128 * 128，然后放入矩阵
        if label:
            # 训练集图像命名方式为 [类别]_[第几张图片].jpg
            y[i] = int(file.split("_")[0]) #图片名分成2个部分，取前面的一个，得到类别作为 y 值
    if label:
      return x, y
    else:
      return x

In [3]:
# 分別將 training set、validation set、testing set 用 readfile 函式讀進來
workspace_dir = 'hw3_data/'
start_time = time.time()
print("Reading data......")
train_x, train_y = readfile(os.path.join(workspace_dir, "training"), True)
print("Size of training data = {}".format(len(train_x)))
val_x, val_y = readfile(os.path.join(workspace_dir, "validation"), True)
print("Size of validation data = {}".format(len(val_x)))
test_x = readfile(os.path.join(workspace_dir, "testing"), False)
print("Size of Testing data = {}".format(len(test_x)))
end_time = time.time()
print("用时：", end_time - start_time)

Reading data......
Size of training data = 1000
Size of validation data = 500
Size of Testing data = 124
用时： 6.275790214538574


# 预处理数据
在 PyTorch 中，我們可以利用 torch.utils.data 的 Dataset 及 DataLoader 來"包裝" data，使後續的 training 及 testing 更為方便。

Dataset 需要 overload 兩個函數：__len__ 及 __getitem__

__len__ 必須要回傳 dataset 的大小，而 __getitem__ 則定義了當程式利用 [ ] 取值時，dataset 應該要怎麼回傳資料。

實際上我們並不會直接使用到這兩個函數，但是使用 DataLoader 在 enumerate Dataset 時會使用到，沒有實做的話會在程式運行階段出現 error。__ getitem __()和__len__() 两个函数必须重写

In [4]:
# 来自torchvision.transforms
# 训练数据做数据增强 (data augmentation)
train_transform = transforms.Compose([ # 处理图片, 用Compose把多个处理步骤整合到一起
    transforms.ToPILImage(), #把数据转换为tensfroms格式
    transforms.RandomHorizontalFlip(), # 隨機將圖片左右镜像
    transforms.RandomRotation(15), # 隨機旋轉圖片
    transforms.ToTensor(), # 將圖片轉成 Tensor，並把數值 normalize 到 [0,1] ,这个格式可以直接输入进神经网络了
])

# 测试数据不需做 数据增强 (data augmentation)
test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(),
])

# 定义ImgDataset类，继承torch.utils.data.Dataset，实现数据读取方式
# torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False,...) 
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        self.y = y
        if y is not None: #将 y 变成一个长向量 64-bit integer (signed)
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index): #取一张图片和对应的分类 y 值
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None: # 判断是否需要 y 值
            Y = self.y[index]
            return X, Y
        else:
            return X

调用上面的定义

In [5]:
batch_size = 128
train_set = ImgDataset(train_x, train_y, train_transform) #训练集
val_set = ImgDataset(val_x, val_y, test_transform) #验证集

# 使用torch.utils.data.DataLoader(), 实现数据的批量读取
#dataset：加载的数据集(Dataset对象); batch_size：batch size; shuffle:：是否将数据打乱
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

# Model

**```torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)```**

每个“filter”实际上是“卷积核”的一个集合, 每个通道都对应一个卷积核, 且这个卷积核是独一无二的, 每一个通道进行卷积运算,最后相加,形成一个filter 的单通道输出
- **in_channels** 每个卷积层中卷积核的数量
- **out_channels** 层中filter的个数，每个filter输出一个矩阵
- **kernel_size** 卷积核的大小，卷积核是一个二维方阵，只需要指定一维
- **步长(Stride)** 滑动filter时，先从输入的左上角开始，每次往左滑动一列或者往下滑动一行逐一计算输出，将每次滑动的行数和列数称为Stride
- **填充（Padding）** 用filter扫描图片时，中心的像素会被扫描多次，而图片边缘的像素只会被扫描一次，导致丢失了图片边界处的信息。有时我们还希望输入和输出的大小应该保持一致。为解决这个问题，可以在矩阵的边界上填充一些值，以增加矩阵的大小 
<img src="hw3_data/pading.gif" width="20%"> 

---

**```torch.nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True)```**

批标准化(Batch Normalization)操作:
- 先对输入进行归一化，$\bar X$为均值，Var(x)为方差, eps 防止分母为 0
- 然后对归一化的结果进行缩放和平移，设置affine=True，即意味着缩放(γ)和平移(β)将被使用
- momentum：动态均值和动态方差所使用的动量 $ x_{new} = (1 - momentum)\times x + momentum \times x_t $ 其中：x是估计的数据 $x_t$是新的观察到的数据
- 默认情况下 $ \gamma =1, bias = 0, momentum = 0.1, eps = 10^{-5} $
- num_features: 输入的层数
$$ y = {{x-\bar X} \over {\sqrt {var(X)+eps}}} \times \gamma + bias $$

---

**```torch.nn.MaxPool2d(kernel_size, stride, padding)```**

- **kernel_size** max pooling的窗口大小
- **stride** max pooling的窗口移动的步长，默认值是kernel_size
- **padding** 输入的每一条边补充0的个数

In [6]:
# import torch.nn as nn
class Classifier(nn.Module):
    def __init__(self): # input 維度 [3, 128, 128]
        super(Classifier, self).__init__() # super() 初始化父类 nn.Module 的属性,避免直接使用父类的名字
        self.cnn = nn.Sequential(
            # 这层有64个filter
            nn.Conv2d(3, 64, 3, 1, 1), # [in_channels = 3, out_channels = 64, kernel_size = 3, stride = 1, padding = 1] 输出[64, 128, 128]
            nn.BatchNorm2d(64), #64 层输入
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # 一张图片用[64, 64, 64]矩阵表示
            
            # 这层有128个filter
            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            # 这层有256个filter
            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            # 这层有512个filter
            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            # 这层有512个filter
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        
        self.fc = nn.Sequential(
            # 一个有序的容器，神经网络模块将按照在传入构造器的顺序依次被添加到计算图中执行
            # 以神经网络模块为元素的有序字典也可以作为传入参数。
            nn.Linear(512*4*4, 1024), #第1层，有 1024 个 neuron，输入 512*4*4 个feature, 输出 1024 维结果
            nn.ReLU(),
            nn.Linear(1024, 512), # 第2层，有 512 个 neuron，输入 1024 个feature, 输出 512 维结果
            nn.ReLU(),
            nn.Linear(512, 11) # 第3层，有 11 个 neuron，输入 512 个feature, 输出 11 维结果，最后一层不需要添加激活函数
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1) # 摊平成1维
        return self.fc(out)

助教写的model，叠很多层时用这种写法
``` python
class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()

    def building_block(indim, outdim):
      return [ nn.Conv2d(indim, outdim, 3, 1, 1),
            nn.BatchNorm2d(outdim),
            nn.ReLU(),
      ]
    def stack_blocks(indim, outdim, block_num):
      layers = building_block(indim, outdim)
      for i in range(block_num - 1):
        layers += building_block(outdim, outdim)
      layers.append(nn.MaxPool2d(2, 2, 0))
      return layers

    cnn_list = []
    cnn_list += stack_blocks(3, 128, 3)
    cnn_list += stack_blocks(128, 128, 3)
    cnn_list += stack_blocks(128, 256, 3)
    cnn_list += stack_blocks(256, 512, 1)
    cnn_list += stack_blocks(512, 512, 1)
    self.cnn = nn.Sequential( * cnn_list)

    dnn_list = [
      nn.Linear(512 * 4 * 4, 1024),
      nn.ReLU(),
      nn.Dropout(p = 0.3),
      nn.Linear(1024, 11),
    ]
    self.fc = nn.Sequential( * dnn_list)

  def forward(self, x):
    out = self.cnn(x)
    out = out.reshape(out.size()[0], -1)
    return self.fc(out)
```

# Training
使用 training set 訓練，並使用 validation set 尋找好的參數

In [9]:
# 初始化
model = Classifier().cuda() # model是Classifier()类
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # optimizer 使用 Adam优化算法 RMSProp(移动窗口) + Momentum（惯性）
num_epoch = 30

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    model.train() # 確保 model 是在 train model (開啟 Dropout 等...)
    for i, data in enumerate(train_loader): # 给train_loader的矩阵一个编号，并组合成一个表
        optimizer.zero_grad() # 用 optimizer 將 model 參數的 gradient 歸零
        train_pred = model(data[0].cuda()) # 利用 model 算出来的機率分佈 這邊實際上就是去呼叫 model 的 forward 函數
        batch_loss = loss(train_pred, data[1].cuda()) # 計算 loss （注意 prediction 跟 label 必須同時在 CPU 或是 GPU 上）
        batch_loss.backward() # 利用 back propagation 算出每個參數的 gradient
        optimizer.step() # 以 optimizer 用 gradient 更新參數值

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()
    
    model.eval() # 不启用 BatchNormalization 和 Dropout 训练完train样本后，生成的模型model用来测试样本
    with torch.no_grad(): #torch.no_grad() 是一个上下文管理器，被该语句 wrap 起来的部分将不会track 梯度
        for i, data in enumerate(val_loader):
            val_pred = model(data[0].cuda())
            batch_loss = loss(val_pred, data[1].cuda())

            val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
            val_loss += batch_loss.item()

        #將結果 print 出來
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, num_epoch, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/val_set.__len__(), val_loss/val_set.__len__()))

RuntimeError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 6.00 GiB total capacity; 4.58 GiB already allocated; 92.50 MiB free; 4.58 GiB reserved in total by PyTorch)

得到好的參數後，我們使用 training set 和 validation set 共同訓練（資料量變多，模型效果較好）

In [8]:
train_val_x = np.concatenate((train_x, val_x), axis=0)
train_val_y = np.concatenate((train_y, val_y), axis=0)
train_val_set = ImgDataset(train_val_x, train_val_y, train_transform)
train_val_loader = DataLoader(train_val_set, batch_size=batch_size, shuffle=True)

In [ ]:
# 和上面一样，省略验证部分
model_best = Classifier().cuda()
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model_best.parameters(), lr=0.001) # optimizer 使用 Adam
num_epoch = 30

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0

    model_best.train()
    for i, data in enumerate(train_val_loader):
        optimizer.zero_grad()
        train_pred = model_best(data[0].cuda())
        batch_loss = loss(train_pred, data[1].cuda())
        batch_loss.backward()
        optimizer.step()

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()

        #將結果 print 出來
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % \
      (epoch + 1, num_epoch, time.time()-epoch_start_time, \
      train_acc/train_val_set.__len__(), train_loss/train_val_set.__len__()))

# 保存模型

In [ ]:
# 保存训练好的完整模型
torch.save(model_best, 'hw3_data/CNN_all.pth')
# 保存训练好的完整模型
torch.save(model_best.state_dict(), 'CNN_parameter_only.pth')

# Testing
利用剛剛 train 好的 model 進行 prediction

In [ ]:
test_set = ImgDataset(test_x, transform=test_transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
model_best.eval()
prediction = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = model_best(data.cuda())
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        for y in test_label:
            prediction.append(y)

In [ ]:
name = ['面包', '奶', '甜品', '蛋', '油炸食品', '肉', '面条', '米饭', '海鲜', '汤', '果蔬']

In [ ]:
#將結果寫入 csv 檔
with open("hw3_data/预测.csv", 'w') as f:
    f.write('Id,Category\n')
    for i, y in  enumerate(prediction):
        f.write('{},{},{}\n'.format(i, y, name[y]))

# 重命名测试集

In [ ]:
# 获取该目录下所有文件，存入列表中
def rename():
    f = os.listdir("hw3_data/testing_prediction")
    for i in range(len(f)):
        oldname = f[i]
        newname = oldname.split(".")[0] + '-'+ name[prediction[i]] + '.jpg'
        # 用os模块中的rename方法对文件改名
        os.rename("hw3_data/testing_prediction/" + oldname, "hw3_data/testing_prediction/" + newname)

